<a href="https://colab.research.google.com/github/AlexxLai/ml0716/blob/master/news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
import zipfile
f = zipfile.ZipFile("drive/My Drive/0724test/chinese_news_trans.zip")
f.extractall("train")
f = zipfile.ZipFile("drive/My Drive/0724test/chinese_news_test.zip")
f.extractall("test")

In [33]:
import glob
import os
import pandas as pd

def getdf(targetdn):
    datas = {
        "path":[],
        "label":[]
    }
    base = ""
    p = os.path.join(base, targetdn, "*")
    for dn in glob.glob(p):
        name = os.path.split(dn)[-1]
        fn = os.path.join(dn, "*.[tT][xX][tT]")
        fns = glob.glob(fn)
        datas["label"] = datas["label"] + [name] * len(fns)
        datas["path"] = datas["path"] + fns
    return pd.DataFrame(datas)

In [37]:
train_df = getdf("train/chinese_news_trans")
test_df = getdf("test/chinese_news_test")

In [ ]:
# train_df
# test_df

In [ ]:
def read_content(fn):
    with open(fn, "r", encoding="utf-8") as f:
        content = f.read()
    return content
train_df["content"] = train_df["path"].apply(read_content)
test_df["content"] = test_df["path"].apply(read_content)
train_df

In [46]:
u = train_df["label"].unique()
trains = {n:i for i, n in enumerate(u)}
trains_r = {i:n for i, n in enumerate(u)}
y_train = train_df["label"].replace(trains)
y_test = test_df["label"].replace(trains)

In [ ]:
import jieba
# 此段的部分是抓取更大的字典集，用以增加正確率
# from urllib.request import urlretrieve
# url = "https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big"
# urlretrieve(url,"dict.txt.big")
# jieba.set_dictionary("dict.txt.big")
p = train_df.iloc[0]["content"]
" ".join(jieba.cut(p))

In [ ]:
def newscut(p):
  return " ".join(jieba.cut(p))
x_train = train_df["content"].apply(newscut)
x_test = test_df["content"].apply(newscut)
x_train

In [52]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
x_train_count = vec.fit_transform(x_train)
x_test_count = vec.transform(x_test)

In [53]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(x_train_count,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [54]:
from sklearn.metrics import accuracy_score
pre = clf.predict(x_test_count)
accuracy_score(pre, y_test)

1.0

In [ ]:
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(y_test,pre)
pd.DataFrame(mat,
        columns = [n+"(預測)" for n in u],
        index = [n+"(真實)" for n in u]
        )

In [64]:
test = input("輸入一則新聞:")
count = vec.transform([newscut(test)]) # 結果必須是Series，因此必須加上中括號轉成list
answer = clf.predict(count)[0]
proba = clf.predict_proba(count)[0] # 機率
for title,p in zip(u,proba):
  print(title,"的機率:",p)
print("答案:",trains_r[answer])
print(proba)

輸入一則新聞:民進黨前秘書長羅文嘉淡出政壇十多年，去年重出江湖，與民進黨前主席卓榮泰合力帶領民進黨打贏大選後，羅文嘉就從執政團隊「裸退」，回憶過去一年執掌黨務，他感慨說，「民進黨已不是我所熟悉的政黨」，民進黨有些處理事情的界線，超過他的理解，「我認為這遲早會出事」。  要更深層檢討政媒關係  「黨政軍退出媒體」一直是民進黨長期的主張，羅文嘉當年率先在中常會發難，要時任民視董事長蔡同榮退出民視，如今，三立董事長林崑海為首的海派掌握媒體資源。羅文嘉說，他立委任內是推動黨政軍退出三台，「我們用政治力量讓政治退出媒體，媒體並沒有因此變成我們想要的媒體。」  羅文嘉說，現在政府或政治要影響媒體不容易，反倒商業力量進入媒體，是帶有政治利益與政治目的，商業掌握媒體資源，再透過媒體來影響政治、參與政治。海派只是局部問題，民進黨一個派系的產生也只是結果，背後更深層的問題是要檢討「政媒關係」，顯然台灣媒體面臨的困境，是向商業利益屈服，要面對媒體老闆影響政治。  民進黨已成世俗化政黨  談到民進黨過去堅守的價值與理念，羅文嘉表示，去年接任民進黨秘書長前，就有人告訴他，「現在的民進黨不一樣了」，但他以自己過去的經驗判斷，總覺得黨內的人事再怎麼變換，有些東西不會變，但歷經一整年的黨務工作，他必須承認，民進黨從政黨文化、派系生態乃至政黨價值，有非常多改變，「當年那個在街頭衝撞、充滿理念的民進黨，如今已成世俗化政黨。」  日前剛落幕的民進黨中常委等黨權力核心改選，羅文嘉說，黨內以前的幹部、老朋友或年輕立委問他，怎麼看待黨職選舉，而他只回應「應該要抑制哪些力量」，給了一點意見。他會這樣做，是想「保存民進黨一股元氣與正氣」。  但即使對民進黨的政治生態與文化感到擔憂，羅文嘉仍認為，面對中國大陸持續的對台壓力，只有民進黨才能守住台灣的現狀，「國民黨不行，其他小黨還沒起來」。
軍事 的機率: 6.091136735748394e-46
交通 的機率: 6.1474456721826215e-77
計算機 的機率: 6.057287806223026e-33
環境 的機率: 1.9750341213089015e-51
藝術 的機率: 9.404793844010334e-32
醫藥 的機率: 1.5660811661413818e-80
教育 的機率: 1.5912508850551988e-15
經濟